In [0]:
%run "../includes/configuration"

In [0]:
demo_df = spark.read.parquet(f"{presentation_folder_path}/race_results").filter("race_year==2020")

In [0]:
from pyspark.sql.functions import count, countDistinct, sum

In [0]:
demo_df.select(count("*")).show()

+--------+
|count(1)|
+--------+
|     340|
+--------+



In [0]:
demo_df.select(countDistinct("race_name")).show()

+-------------------------+
|count(DISTINCT race_name)|
+-------------------------+
|                       17|
+-------------------------+



In [0]:
demo_df.filter("driver_name = 'Lewis Hamilton'").select(sum("points")).show()

+-----------+
|sum(points)|
+-----------+
|      347.0|
+-----------+



In [0]:
demo_df.filter("driver_name = 'Lewis Hamilton'").select(sum("points"), countDistinct("race_name")).show()

+-----------+-------------------------+
|sum(points)|count(DISTINCT race_name)|
+-----------+-------------------------+
|      347.0|                       16|
+-----------+-------------------------+



In [0]:
demo_df.groupBy("driver_name").agg(sum("points"), countDistinct("race_name")).show()

+------------------+-----------+----------------+
|       driver_name|sum(points)|count(race_name)|
+------------------+-----------+----------------+
|       Jack Aitken|        0.0|               1|
|      Daniil Kvyat|       32.0|              17|
|   Kevin Magnussen|        1.0|              17|
|      Sergio Pérez|      125.0|              15|
|      Carlos Sainz|      105.0|              17|
|    Kimi Räikkönen|        4.0|              17|
|   Romain Grosjean|        2.0|              15|
|   Charles Leclerc|       98.0|              17|
|   Alexander Albon|      105.0|              17|
|      Lance Stroll|       75.0|              16|
|      Pierre Gasly|       75.0|              17|
|    Lewis Hamilton|      347.0|              16|
|   Nico Hülkenberg|       10.0|               3|
|  Daniel Ricciardo|      119.0|              17|
|   Valtteri Bottas|      223.0|              17|
|Antonio Giovinazzi|        4.0|              17|
|      Lando Norris|       97.0|              17|


In [0]:
demo_df.groupBy("race_year", "driver_name").agg(sum("points"), countDistinct("race_name")).show()

+---------+------------------+-----------+----------------+
|race_year|       driver_name|sum(points)|count(race_name)|
+---------+------------------+-----------+----------------+
|     2020|      Daniil Kvyat|       32.0|              17|
|     2020|   Kevin Magnussen|        1.0|              17|
|     2020|Antonio Giovinazzi|        4.0|              17|
|     2020|   Nico Hülkenberg|       10.0|               3|
|     2020|   Romain Grosjean|        2.0|              15|
|     2020|   Charles Leclerc|       98.0|              17|
|     2020|      Esteban Ocon|       62.0|              17|
|     2020| Pietro Fittipaldi|        0.0|               2|
|     2020|  Sebastian Vettel|       33.0|              17|
|     2020|  Daniel Ricciardo|      119.0|              17|
|     2020|    Kimi Räikkönen|        4.0|              17|
|     2020|    George Russell|        3.0|              17|
|     2020|    Max Verstappen|      214.0|              17|
|     2020|   Valtteri Bottas|      223.

In [0]:
demo_df_win = spark.read.parquet(f"{presentation_folder_path}/race_results").filter("race_year in (2019, 2020)")

In [0]:
demo_grouped_df = demo_df_win.groupBy("race_year", "driver_name").agg(sum("points"), countDistinct("race_name"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

In [0]:
driverRankSpec = Window.partitionBy("race_year").orderBy(desc("sum(points)"))
demo_grouped_df.withColumn("rank", rank().over(driverRankSpec)).show(100)

+---------+------------------+-----------+----------------+----+
|race_year|       driver_name|sum(points)|count(race_name)|rank|
+---------+------------------+-----------+----------------+----+
|     2019|    Lewis Hamilton|      413.0|              21|   1|
|     2019|   Valtteri Bottas|      326.0|              21|   2|
|     2019|    Max Verstappen|      278.0|              21|   3|
|     2019|   Charles Leclerc|      264.0|              21|   4|
|     2019|  Sebastian Vettel|      240.0|              21|   5|
|     2019|      Carlos Sainz|       96.0|              21|   6|
|     2019|      Pierre Gasly|       95.0|              21|   7|
|     2019|   Alexander Albon|       92.0|              21|   8|
|     2019|  Daniel Ricciardo|       54.0|              21|   9|
|     2019|      Sergio Pérez|       52.0|              21|  10|
|     2019|      Lando Norris|       49.0|              21|  11|
|     2019|    Kimi Räikkönen|       43.0|              21|  12|
|     2019|   Nico Hülken